<h1>Globals</h1>

In [93]:
from __future__ import print_function
from datetime import datetime
from git import Repo, Git
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import os.path
import re, string, ntpath, keyword, json, codecs


skip_projects = [ "androidannotations",
                  "bigbluebutton",
                  "cassandra",
                  "elasticsearch",
                  "hibernate-orm",
                  "liferay-portal",
                  "netty",
                  "platform_frameworks_base",
                  "spring-framework",
                  "wildfly"]
tag_names = ["2016-06","2016-01","2015-06","2015-01","2014-06","2014-01","2013-06","2013-01",
"2012-06","2012-01","2011-06","2011-01","2010-06","2010-01","2009-06","2009-01",
"2008-06","2008-01","2007-06","2007-01","2006-06","2006-01","2005-06","2005-01",
"2004-06","2004-01","2003-06","2003-01"]

project_type_map = {"java" : ".java",
                    "python" : ".py",
                    "ruby" : ".rb",
                    "php" : ".php",
                    "javascript" : ".js"}

def load_config(config_file):
    """
    Load projects configuration file.
    """
    with open(config_file) as data_file:    
        config_data = json.load(data_file)
    return config_data

base_dir = "/home/hshahin/workspaces/Spring2016_SE_Project"
data_dir = os.path.join(base_dir , "data")
config_file = "projects_config.json"
config_data = load_config(os.path.join(base_dir , config_file))

<h1>Creating tags functions</h1>

In [94]:
def get_date_time(epoch):
    '''
    convert epoch to date_time
    '''
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch))

def tag_exists(path, tag_name):
    repo = Repo(path)
    
    return True if tag_name in repo.tags else False

def get_epoch(year, month='01'):
    """
    calculate the epoch of first day of a year-month
    """
    pattern = '%Y.%m.%d %H:%M:%S'
    return int(time.mktime(time.strptime(str(year) + '.' + str(month) + '.01 00:00:00', pattern)))


def create_tags(path):
    '''
    takes repo path and creates tags for first commit in Jan and Jun. for every year
    # get the list of commits
    # get the latest commit date
    # current_year is the year from that date
    # loop through the list of commit to find the commit having a date equal or just after 1/1/current_year
    # once found create a tage with the current_year name on it AND
    # subtract 1 from the year and continue.

    '''
    repo = Repo(path)

    # get the list of commits
    commits = list(repo.iter_commits())

    # get the latest commit date, current_year is the year from that date
    current_year = datetime.fromtimestamp(commits[0].committed_date).year


    for idx, commit in enumerate(commits):
        # time.sleep(2)
        # print(commits[idx].hexsha)

        current_year_01 = str(current_year)+'-01'
        current_year_06 = str(current_year)+'-06'

        try:
            if get_epoch(current_year, '01') > commit.committed_date and \
                int(time.time()) > get_epoch(current_year, '01')  and \
                idx !=0:
                if str(current_year_01) not in repo.tags and idx != 0:
                    print(commits[idx-1].hexsha+' '+get_date_time(commits[idx-1].committed_date)+' '+current_year_01)
                    past = repo.create_tag(current_year_01, ref=commits[idx-1],
                                      message="This is a tag to mark the first commit in year %s" % current_year_01)
                current_year = datetime.fromtimestamp(commit.committed_date).year

            if get_epoch(current_year, '06') > commit.committed_date and \
                int(time.time()) > get_epoch(current_year, '06') and \
                idx != 0:
                if str(current_year_06) not in repo.tags:
                    print(commits[idx-1].hexsha+' '+get_date_time(commits[idx-1].committed_date)+' '+current_year_06)
                    past = repo.create_tag(current_year_06, ref=commits[idx-1],
                                      message="This is a tag to mark the first commit in year %s" % current_year_06)
        except AttributeError:
            pass

def checkout_tag(path, tag_name):
    '''
    checks out a tag if it exists
    '''
    repo = Repo(path)
    git = Git(path)
    if tag_name in repo.tags:
        git.checkout(tag_name)

def delete_tags(path):
    '''
    remove all tags in a given repo
    '''

    repo = Repo(path)

    for tag in repo.tags:
        repo.delete_tag(tag)

<h1>Create tags every 6 months for each repo</h1>

In [74]:
for project_name, project_type in config_data.items():
    print("Processing project: " + project_name )
    t0 = time.time()
    delete_tags(os.path.join(data_dir, project_name))
    create_tags(os.path.join(data_dir, project_name))
    print("Project: " + project_name + " taged in %0.3fs." % (time.time() - t0))

Processing project: symfony
169d5513a615e2a6346c9f4566c0ef86579f8d79 2016-01-01 10:10:52 2016-01
fb9e5bfc690876ec6e22699b341596e1e47c365a 2015-06-01 04:01:07 2015-06
5a121df42be456f0dd0e262f50618b12f1293523 2015-01-01 07:56:52 2015-01
1793292ae960e415029635143f34390a326b4e31 2014-06-01 11:15:00 2014-06
7f9a366529c7996ed94b9460013940d590ae7a61 2014-01-01 02:52:14 2014-01
6d55422fd07947db0f920360492ca22336a48a74 2013-06-02 07:57:02 2013-06
13f0a807b101b6fa20664e9de49875cd8b8b99f0 2013-01-02 04:30:52 2013-01
e08252897a89d3112c177bffab8b964cebbfca90 2012-06-01 01:45:48 2012-06
fae715798aa2f5a08d33fa059f8a8e3e0ce25822 2012-01-01 14:48:21 2012-01
3e68eb61a50de6c279238cb4a8bc79147f34d076 2011-06-01 00:35:31 2011-06
d8b8ae0608b8b9933865a84f8d3acb6e044bbfb4 2011-01-02 04:41:00 2011-01
1a3790a636f78ec7c5b08e6981672cf323428c5c 2010-06-01 04:03:50 2010-06
Project: symfony taged in 18.782s.
Processing project: netty
bc2559ceb1fba6b1f97d99494be90e83b3ead6e7 2016-01-04 02:47:00 2016-01
c7b164b4090b66

<h1>Preprocesing functions</h1>

In [95]:
# Python keywords
python_keywords = keyword.kwlist

# Java keywords from https://docs.oracle.com/javase/tutorial/java/nutsandbolts/_keywords.html
java_keywords = ["abstract","continue","for","new","switch","assert","default","goto","package","synchronized",
                 "boolean","do","if","private","this","break","double","implements","protected","throw",
                 "byte","else","import","public","throws","case","enum","instanceof","return","transient","catch",
                 "extends","int","short","try","char","final","interface","static","void","class","finally","long",
                 "strictfp","volatile","const","float","native","super","while"]

# Ruby keywords from http://docs.ruby-lang.org/en/2.2.0/keywords_rdoc.html
ruby_keywords = ["__ENCODING__","__LINE__","__FILE__","BEGIN","END","alias","and","begin","break",
                 "case","class","def","defined?","do","else","elsif","end","ensure","false","for","if",
                 "in","module","next","nil","not","or","redo","rescue","retry","return","self","super",
                 "then","true","undef","unless","until","when","while","yield"]

# PHP keywords form http://php.net/manual/en/reserved.keywords.php
php_keywords = ["__halt_compiler","abstract","and","array","as","break","callable","case","catch","class","clone",
                "const","continue","declare","default","die","do","echo","else","elseif","empty","enddeclare",
                "endfor","endforeach","endif","endswitch","endwhile","eval","exit","extends","final","finally",
                "for","foreach","function","global","goto","if","implements","include","include_once","instanceof",
                "insteadof","interface","isset","list","namespace","new","or","print","private","protected",
                "public","require","require_once","return","static","switch","throw","trait","try","unset","use",
                "var","while","xor","yield"]

all_keywords = python_keywords + java_keywords + ruby_keywords + php_keywords


def path_leaf(path):
    head, tail = ntpath.split(path)
    return head, tail

# split camel case tokens
_underscorer1 = re.compile(r'(.)([A-Z][a-z]+)')
_underscorer2 = re.compile('([a-z0-9])([A-Z])')


def camel_to_spaces(s):
    """
    convert camel case into spaces seperated
    """
    subbed = _underscorer1.sub(r'\1 \2', s)
    return _underscorer2.sub(r'\1 \2', subbed).lower()

def snake_to_spaces(snake_cased_str):
    """
    convert snake case into spaces seperated
    """
    separator = "_"
    components = snake_cased_str.split(separator)
    if components[0] == "":
      components = components[1:]
    if components[-1] == "":
      components = components[:-1]
    if len(components) > 1:
      spaced_str = components[0].lower()
      for x in components[1:]:
        spaced_str += " " + x.lower()
    else:
      spaced_str = components[0]
    return spaced_str


def file_preprocessing(input_file, output_file):
    """
    - replace punctuations with spaces
    - stemming
    - camel to spaces and snake to spaces
    - remove language spesific keywords
    - write the entire project snapshot into one file under project root folder
    """
    # print("processing file " + input_file)
    # replace the punctuations with space
    replace_punctuation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    # stemming
    stemmer = PorterStemmer()

    with open(input_file, 'r') as inFile, open(output_file,'w') as outFile:
        for line in inFile:
            # replace punctuations
            # convert camel case into space separated
            # convert snake case into space separated             
            # remove language keywords
            line_witout_puncs = ' '.join([snake_to_spaces(camel_to_spaces(word)) \
                                          for word in line.translate(replace_punctuation).split() \
                                          if len(word) >=4 and \
                                          word not in stopwords.words('english') and \
                                          word not in all_keywords])

            # stemming
            singles = []
            for plural in line_witout_puncs.split():
                try:
                    singles.append(stemmer.stem(plural))
                except UnicodeDecodeError:
                    print(plural) 

            line_stemmed = ' '.join(singles)
            print(line_stemmed, file=outFile)


def return_file_type(project_path, file_type):
    project_files = [os.path.join(root, name)
             for root, dirs, files in os.walk(project_path)
             for name in files
             if name.endswith((file_type))]
    return project_files[:10]

def project_preprocessing(project_path, file_type, tag_name):
    # print ("processing project "+ project_path)
    # process project source code files and save each file as .*.proc 
    project_files = return_file_type (project_path, file_type)
    for source_file in project_files:
        head, tail = path_leaf(source_file)
        proc_file = os.path.join(head , tail + '.proc')
        file_preprocessing(source_file, proc_file)
        
    # concatenate all processed project files into one file under root directory
    project_proc_files = return_file_type (project_path, file_type + '.proc')
    with open(os.path.join(project_path , tag_name + "-concatenated.out"), 'w') as outfile:
        for fname in project_proc_files:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

<h1>Run preprocessing</h1>

In [ ]:
for tag_name in tag_names:
    for project_name, project_type in config_data.items():
        project_path = os.path.join(data_dir, project_name)
        if tag_exists(project_path, tag_name):
            # check out year-month tag
            checkout_tag(project_path, tag_name)
            t0 = time.time()
            # processing project if tag exists
            project_preprocessing(project_path, project_type_map[str(project_type)], tag_name)
            print("processing project: " + project_name + "\t\t tag " 
                  + tag_name + " done in %0.3fs." % (time.time() - t0))

processing project: symfony		 tag 2016-01 done in 1.354s.
processing project: netty		 tag 2016-01 done in 1.442s.
processing project: androidannotations		 tag 2016-01 done in 0.577s.
processing project: express		 tag 2016-01 done in 0.297s.
processing project: nodal		 tag 2016-01 done in 0.509s.
processing project: laravel		 tag 2016-01 done in 0.367s.
processing project: django		 tag 2016-01 done in 1.782s.
processing project: mean		 tag 2016-01 done in 0.444s.
processing project: ninja		 tag 2016-01 done in 0.562s.
processing project: platform_frameworks_base		 tag 2016-01 done in 6.236s.
processing project: sinatra		 tag 2016-01 done in 1.799s.
processing project: pakyow		 tag 2016-01 done in 0.191s.
processing project: elasticsearch		 tag 2016-01 done in 3.072s.
processing project: meteor		 tag 2016-01 done in 3.217s.


<h1>Topic Modeling</h1>

In [4]:
from __future__ import print_function
import os
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_features = 1000
n_topics = 10
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

processed_path = []
for project_name, project_type in config_data.items():
    processed_path.append(os.path.join(data_dir, project_name, "concatenated.out"))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=0.05, max_features=n_features, stop_words='english')

t0 = time()
tf = tf_vectorizer.fit_transform(processed_path)
print("done in %0.3fs." % (time() - t0))

print("Fitting LDA models with tf features")
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf features for LDA...
done in 0.003s.
Fitting LDA models with tf features
done in 0.021s.

Topics in LDA model:
Topic #0:
androidannotations cassandra portal liferay orm hibernate bigbluebutton platform_frameworks_base netty framework spring elasticsearch wildfly
Topic #1:
bigbluebutton elasticsearch framework netty orm androidannotations wildfly spring hibernate platform_frameworks_base portal cassandra liferay
Topic #2:
wildfly spring hibernate elasticsearch framework bigbluebutton netty liferay orm cassandra androidannotations portal platform_frameworks_base
Topic #3:
platform_frameworks_base wildfly liferay portal netty bigbluebutton androidannotations framework elasticsearch spring cassandra hibernate orm
Topic #4:
orm hibernate wildfly platform_frameworks_base spring cassandra bigbluebutton elasticsearch androidannotations portal liferay netty framework
Topic #5:
liferay framework androidannotations portal orm spring bigbluebutton elasticsearch hibernate platform_fram

In [5]:
def print_full(x):
    pd.set_option('display.max_cols')
    print(x)
    pd.reset_option('display.max_rows')

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
df = pd.DataFrame(lda.components_)
df.columns = tf_feature_names
df
# print_full(df)


,androidannotations,bigbluebutton,cassandra,elasticsearch,framework,hibernate,liferay,netty,orm,platform_frameworks_base,portal,spring,wildfly
0,1.160414,0.780085,1.119434,0.682189,0.743249,0.780858,1.029484,0.759556,0.783337,0.775150,1.086674,0.735939,0.581064
1,0.799101,1.202158,0.650373,0.951796,0.862462,0.688270,0.616478,0.843549,0.841355,0.677368,0.652624,0.714402,0.719431
2,0.639969,0.735402,0.687752,0.779103,0.748662,0.782179,0.705713,0.724742,0.693375,0.632709,0.638909,0.784749,0.859564
3,0.760091,0.780247,0.702219,0.728370,0.754778,0.669530,0.869986,0.784969,0.645014,1.113746,0.789782,0.721883,1.013000
4,0.776832,0.803112,0.809359,0.780224,0.662507,1.097508,0.721147,0.697893,1.168896,0.818353,0.731458,0.810408,0.845535
5,0.819597,0.778321,0.673944,0.772531,0.851114,0.719631,0.893120,0.696347,0.790476,0.705611,0.800823,0.779772,0.674371
6,0.762848,0.797694,0.686569,0.834690,1.020041,0.882066,0.743592,0.703694,0.670713,0.719448,0.715216,1.173972,0.665509
7,0.813620,0.680634,0.644463,1.009442,0.774040,0.678330,0.801214,0.694047,0.701886,0.725362,0.655592,0.796950,0.640738
8,0.673608,0.754491,0.661026,0.770228,0.789278,0.738344,0.919016,0.851920,0.724283,0.733501,0.833331,0.799219,0.748946
9,0.698543,0.771175,0.743619,0.818673,0.773928,0.684413,0.721399,1.201284,0.747644,0.717636,0.786148,0.762099,0.767659


In [7]:
processed_path = []
for project_name, project_type in config_data.items():
    processed_path.append(os.path.join(data_dir, project_name, "concatenated.out"))

In [9]:
print(processed_path)

['/home/hshahin/workspaces/Spring2016_SE_Project/data/platform_frameworks_base/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/bigbluebutton/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/cassandra/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/spring-framework/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/netty/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/androidannotations/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/liferay-portal/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/elasticsearch/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/wildfly/concateneated.out', '/home/hshahin/workspaces/Spring2016_SE_Project/data/hibernate-orm/concateneated.out']
